# Angleichung der Label Bezeichnungnen beider Datensätze

In [49]:
import os 
import pandas as pd
import logging
import multiprocessing as mp

In [50]:
# Logging Parameter
logging.basicConfig(
    filename='22_angleichung_labels.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Verzeichnis der CSV-Dateien
verzeichnis_ids17 = '../01_Datensaetze/improved_cic-ids-2017'
verzeichnis_ids18 = '../01_Datensaetze/improved_cse-cic-ids-2018'

nrows = None
chunksize = 500000

# Mapping von CICIDS2017-Bezeichnungen zu CICIDS2018-Bezeichnungen
mapping_2017_to_2018 = {
    'Botnet': 'Botnet Ares',
    'Botnet - Attempted': 'Botnet Ares - Attempted',
    'DDoS': 'DDoS-LOIC-HTTP',
    'Infiltration - Portscan': 'Infiltration - NMAP Portscan',
    'FTP-Patator - Attempted': 'FTP-BruteForce - Attempted',
    'SSH-Patator': 'SSH-BruteForce',
    'Web Attack - SQL Injection': 'Web Attack - SQL',
    'Web Attack - SQL Injection - Attempted': 'Web Attack - SQL - Attempted'
}

In [33]:
def get_labels_from_file(verzeichnis, datei):
    if datei.endswith('.csv'):
        pfad_zur_datei = os.path.join(verzeichnis, datei)
        try:
            df = pd.read_csv(pfad_zur_datei, nrows=nrows)
            found_labels = df['Label'].unique()
            logging.info(f"Successfully read {pfad_zur_datei}. Found labels: {found_labels}")
            return found_labels
        except Exception as e:
            logging.error(f"Error reading {pfad_zur_datei}: {e}")
            return []
    return []

def get_labels_from_dir(verzeichnis):
    try:
        logging.info(f"Start processing directory {verzeichnis}")
        with mp.Pool(processes=2) as pool:
            results = pool.starmap(get_labels_from_file, [(verzeichnis, datei) for datei in os.listdir(verzeichnis)])

        labels = set()
        for result in results:
            labels.update(result)
        
        logging.info(f"Successfully processed directory {verzeichnis}")
        return labels
    except Exception as e:
        logging.error(f"Error processing directory {verzeichnis}: {e}")
        return set()

In [34]:
labels_ids17 = get_labels_from_dir(verzeichnis_ids17)

In [35]:
labels_ids18 = get_labels_from_dir(verzeichnis_ids18)

In [36]:
labels_ids17

{'BENIGN',
 'Botnet Ares',
 'Botnet Ares - Attempted',
 'DDoS-LOIC-HTTP',
 'DoS GoldenEye',
 'DoS GoldenEye - Attempted',
 'DoS Hulk',
 'DoS Hulk - Attempted',
 'DoS Slowloris',
 'DoS Slowloris - Attempted',
 'FTP-Patator - Attempted',
 'Infiltration - NMAP Portscan',
 'SSH-Bruteforce',
 'Web Attack - Brute Force',
 'Web Attack - Brute Force - Attempted',
 'Web Attack - SQL Injection',
 'Web Attack - SQL Injection - Attempted',
 'Web Attack - XSS',
 'Web Attack - XSS - Attempted'}

In [37]:
labels_ids18

{'BENIGN',
 'Botnet Ares',
 'Botnet Ares - Attempted',
 'DDoS-LOIC-HTTP',
 'DoS GoldenEye',
 'DoS GoldenEye - Attempted',
 'DoS Hulk',
 'DoS Hulk - Attempted',
 'DoS Slowloris',
 'DoS Slowloris - Attempted',
 'FTP-BruteForce - Attempted',
 'Infiltration - NMAP Portscan',
 'SSH-BruteForce',
 'Web Attack - Brute Force',
 'Web Attack - Brute Force - Attempted',
 'Web Attack - SQL',
 'Web Attack - SQL - Attempted',
 'Web Attack - XSS',
 'Web Attack - XSS - Attempted'}

In [38]:
labels_ids17 == labels_ids18

False

Die Angriffe unterscheiden sich teilweise. Nach genauer Analyse werden die Datensätze aneinander angepasst. Dabei werden Angriffe entfernt, zusammengefasst oder Bezeichnungen angepasst. 


## Anpassung der Bezeichnungen der Labels

In [51]:
def relabel_file(file_path):
    temp_file = file_path + '.tmp'
    try:
        # Lade die CSV-Datei in Chunks
        chunks = pd.read_csv(file_path, chunksize=chunksize)

        # Verarbeite jeden Chunk und speichere in eine temporäre Datei
        for i, chunk in enumerate(chunks):
            # Prüfe, ob es eine Spalte mit den Angriffsnamen gibt (z.B. 'Label')
            if 'Label' in chunk.columns:
                # Ersetze die Labels basierend auf dem Mapping
                chunk['Label'] = chunk['Label'].map(mapping_2017_to_2018).fillna(chunk['Label'])

                # Speichere den Chunk in die temporäre Datei
                if i == 0:
                    chunk.to_csv(temp_file, mode='w', header=True, index=False)
                else:
                    chunk.to_csv(temp_file, mode='a', header=False, index=False)
            else:
                logging.warning(f"Keine 'Label'-Spalte gefunden in {file_path}")

        # Ersetze die Originaldatei durch die temporäre Datei
        os.replace(temp_file, file_path)

        logging.info(f"Datei verarbeitet und gespeichert: {file_path}")
    
    except Exception as e:
        logging.error(f"Fehler beim Verarbeiten der Datei {file_path}: {e}")
        if os.path.exists(temp_file):
            os.remove(temp_file)  # Entferne die temporäre Datei im Fehlerfall

def relabel_directory(directory):
    try:
        logging.info(f"Start processing directory {directory}")
        with mp.Pool(processes=5) as pool:
            pool.map(relabel_file, [os.path.join(directory, datei) for datei in os.listdir(directory) if datei.endswith('.csv')])

        logging.info(f"Successfully processed directory {directory}")
    except Exception as e:
        logging.error(f"Error processing directory {directory}: {e}")


In [52]:
relabel_directory(verzeichnis_ids17)

In [53]:
new_labels_ids17 = get_labels_from_dir(verzeichnis_ids17)

In [54]:
new_labels_ids17

{'BENIGN',
 'Botnet Ares',
 'Botnet Ares - Attempted',
 'DDoS-LOIC-HTTP',
 'DoS GoldenEye',
 'DoS GoldenEye - Attempted',
 'DoS Hulk',
 'DoS Hulk - Attempted',
 'DoS Slowhttptest',
 'DoS Slowhttptest - Attempted',
 'DoS Slowloris',
 'DoS Slowloris - Attempted',
 'FTP-BruteForce - Attempted',
 'FTP-Patator',
 'Heartbleed',
 'Infiltration',
 'Infiltration - Attempted',
 'Infiltration - NMAP Portscan',
 'Portscan',
 'SSH-BruteForce',
 'SSH-Patator - Attempted',
 'Web Attack - Brute Force',
 'Web Attack - Brute Force - Attempted',
 'Web Attack - SQL',
 'Web Attack - SQL - Attempted',
 'Web Attack - XSS',
 'Web Attack - XSS - Attempted'}

Die Bezeichnungen sind angepasst.

## Angriffe entfernen, die nicht in beiden Datensätzen vorhanden sind.

In [55]:
remove_in_ids17 = ('DoS Slowhttptest', 'DoS Slowhttptest - Attempted', 'FTP-Patator', 'Heartbleed', 'Infiltration', 'Infiltration - Attempted', 'Portscan', 'SSH-Patator - Attempted')
remove_in_ids18 = ('DDoS-HOIC', 'DDoS-LOIC-UDP', 'DDoS-LOIC-UDP - Attempted', 'Infiltration - Communication Victim Attacker', 'Infiltration - Dropbox Download', 'Infiltration - Dropbox Download - Attempted')

In [56]:
chunksize = 500000  # Definiere deine Chunk-Größe hier

# Funktion, um eine CSV-Datei zu verarbeiten und Zeilen mit bestimmten Labels zu entfernen
def remove_rows_with_labels(file_path, labels_to_remove):
    temp_file = file_path + '.tmp'  # Temporäre Datei, in die wir schreiben
    try:
        # Lade die CSV-Datei in Chunks
        chunks = pd.read_csv(file_path, chunksize=chunksize)

        # Verarbeite jeden Chunk und speichere in die temporäre Datei
        for i, chunk in enumerate(chunks):
            # Entferne die Zeilen mit den angegebenen Labels
            chunk = chunk[~chunk['Label'].isin(labels_to_remove)]

            # Speichere den Chunk ohne die entfernten Zeilen in die temporäre Datei
            if i == 0:
                chunk.to_csv(temp_file, mode='w', header=True, index=False)
            else:
                chunk.to_csv(temp_file, mode='a', header=False, index=False)

        # Ersetze die Originaldatei durch die temporäre Datei
        os.replace(temp_file, file_path)

        logging.info(f"Zeilen mit Labels '{labels_to_remove}' entfernt und Datei überschrieben: {file_path}")
    
    except Exception as e:
        logging.error(f"Fehler beim Verarbeiten der Datei {file_path}: {e}")
        if os.path.exists(temp_file):
            os.remove(temp_file)  # Entferne die temporäre Datei im Fehlerfall

# Funktion, um alle CSV-Dateien im Verzeichnis zu verarbeiten
def remove_labels_from_all_files(directory, labels_to_remove):
    # Hole alle CSV-Dateien im Verzeichnis
    csv_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.csv')]

    # Nutze Multiprocessing, um die Dateien parallel zu verarbeiten
    with mp.Pool(processes=4) as pool:
        pool.starmap(remove_rows_with_labels, [(file, labels_to_remove) for file in csv_files])


In [57]:
remove_labels_from_all_files(verzeichnis_ids17, remove_in_ids17)

In [58]:
new_labels_ids17 = get_labels_from_dir(verzeichnis_ids17)

In [59]:
new_labels_ids17

{'BENIGN',
 'Botnet Ares',
 'Botnet Ares - Attempted',
 'DDoS-LOIC-HTTP',
 'DoS GoldenEye',
 'DoS GoldenEye - Attempted',
 'DoS Hulk',
 'DoS Hulk - Attempted',
 'DoS Slowloris',
 'DoS Slowloris - Attempted',
 'FTP-BruteForce - Attempted',
 'Infiltration - NMAP Portscan',
 'SSH-BruteForce',
 'Web Attack - Brute Force',
 'Web Attack - Brute Force - Attempted',
 'Web Attack - SQL',
 'Web Attack - SQL - Attempted',
 'Web Attack - XSS',
 'Web Attack - XSS - Attempted'}

In [25]:
remove_labels_from_all_files(verzeichnis_ids18, remove_in_ids18)

In [26]:
new_labels_ids18 = get_labels_from_dir(verzeichnis_ids18)

In [45]:
new_labels_ids18

{'BENIGN',
 'Botnet Ares',
 'Botnet Ares - Attempted',
 'DDoS-LOIC-HTTP',
 'DoS GoldenEye',
 'DoS GoldenEye - Attempted',
 'DoS Hulk',
 'DoS Hulk - Attempted',
 'DoS Slowloris',
 'DoS Slowloris - Attempted',
 'FTP-BruteForce - Attempted',
 'Infiltration - NMAP Portscan',
 'SSH-BruteForce',
 'Web Attack - Brute Force',
 'Web Attack - Brute Force - Attempted',
 'Web Attack - SQL',
 'Web Attack - SQL - Attempted',
 'Web Attack - XSS',
 'Web Attack - XSS - Attempted'}

In [60]:
new_labels_ids18 == new_labels_ids17

True